In [ ]:
# !sed -i 's/transformers.generation_utils/transformers.generation/' /usr/local/lib/python3.12/dist-packages/openprompt/pipeline_base.py

In [ ]:
# !pip install transformers==4.24.0

In [1]:
# Set device
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
# ==============================
# Imports
# ==============================
import torch
import pandas as pd
import random
import numpy as np
from collections import Counter
from transformers import XLMRobertaTokenizer, XLMRobertaForMaskedLM
from openprompt.prompts import ManualTemplate, ManualVerbalizer
from openprompt.prompts import SoftVerbalizer
from openprompt.prompts import AutomaticVerbalizer
from openprompt.data_utils import InputExample
from openprompt.plms import load_plm
from openprompt import PromptForClassification, PromptDataLoader
from torch.optim import AdamW
from sklearn.metrics import classification_report
from torch.utils.data import DataLoader, Sampler

C:\Users\stdFurqan\anaconda3\envs\py310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:


# ==============================
# Set random seeds for reproducibility
# ==============================
# 💡 Added this block to ensure consistent results across runs
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [4]:
# ==============================
# Balanced Batch Sampler
# ==============================
class BalancedBatchSampler(Sampler):
    def __init__(self, dataset, batch_size):
        """
        dataset: list of InputExample
        batch_size: total batch size (must be divisible by number of classes)
        """
        self.dataset = dataset
        self.labels = [ex.label for ex in dataset]
        self.classes = list(sorted(set(self.labels)))
        self.num_classes = len(self.classes)
        assert batch_size % self.num_classes == 0, "Batch size must be divisible by number of classes"
        self.batch_size_per_class = batch_size // self.num_classes

    def __iter__(self):
        class_indices = {c: np.where(np.array(self.labels) == c)[0].tolist() for c in self.classes}
        for c in self.classes:
            np.random.shuffle(class_indices[c])

        num_batches = min(len(class_indices[c]) // self.batch_size_per_class for c in self.classes)

        for i in range(num_batches):
            batch = []
            for c in self.classes:
                start = i * self.batch_size_per_class
                end = start + self.batch_size_per_class
                batch.extend(class_indices[c][start:end])
            np.random.shuffle(batch)
            yield batch

    def __len__(self):
        return min(len(np.where(np.array(self.labels) == c)[0]) // self.batch_size_per_class for c in self.classes)

In [5]:

# Step 1: Training data (16-shot) ha,m and spam
train_dataset = [

    InputExample(guid=0, text_a="پروفیسر نے بات کی ہم سے۔", label=0),
    InputExample(guid=1, text_a="پروفیسر نے بات کی ہم سے۔", label=0),
    InputExample(guid=2, text_a="م خود سے چیخ رہے تھا ۔", label=0),
    InputExample(guid=3, text_a="ہم نے ہیری کو گھبرا کر پکارا تھا۔", label=0),
    InputExample(guid=4, text_a="ہیری نے خود کو کھانسی دے دیا۔", label=0),
    InputExample(guid=5, text_a="ہیری نے ہمیں کھانسی دے کر فٹ کر دیتا تھا ۔", label=0),
    InputExample(guid=6, text_a="انہوں نے اسے بنا کر غصہ دلاتا تھا ۔", label=0),
    InputExample(guid=7, text_a="انہوں نے اسے صدر بنا کر بنا دیتے تھا ۔", label=0),
    InputExample(guid=8, text_a="انہوں نے اسے تھکا دیتا تھا ۔", label=0),
    InputExample(guid=9, text_a="گاڑی سڑک پر ہارن مارتی تھی کو۔", label=0),
    InputExample(guid=10, text_a="کتا کمرے سے باہر بھونکا دیا۔", label=0),
    InputExample(guid=11, text_a="ڈائن غائب ہو کر جنگل میں گئی تھی۔", label=0),
    InputExample(guid=12, text_a="عمارت اونچی اور اونچی ہیں ۔", label=0),
    InputExample(guid=13, text_a="یہ عمارت اونچی اور اونچی ہیں ۔", label=0),
    InputExample(guid=14, text_a="اس عمارت کو اس سے زیادہ مل گیا ہے تھے ۔", label=0),
    InputExample(guid=15, text_a="یہ عمارت اس سے زیادہ ہیں ۔", label=0),

    
    InputExample(guid=16, text_a="ہمارے دوست اس تجزیے کو نہیں خریدیں گے - اگلے تجزیے کو چھوڑ دیں جو ہم تجویز کرتے ہیں ۔", label=1),
    InputExample(guid=17, text_a="ایک اور فرضی عام کاری اور میں ہار مان رہا ہوں ۔", label=1),
    InputExample(guid=18, text_a="ایک اور فرضی عام کاری یا میں ترک کر رہا ہوں ۔", label=1),
    InputExample(guid=19, text_a="ہم فعل کا جتنا زیادہ مطالعہ کرتے ہیں وہ اتنے ہی پاگل ہوتے جاتے ہیں ۔", label=1),
    InputExample(guid=20, text_a="روز روز حقائق مبہم ہوتے جا رہے ہیں ۔", label=1),
    InputExample(guid=21, text_a="میں آپ کو ایک مشروب ٹھیک کر دوں گا ۔", label=1),
    InputExample(guid=22, text_a="فریڈ نے پودوں کو چپٹا پانی دیا ۔", label=1),
    InputExample(guid=23, text_a="بل کو کھانسی آنے پر ریستوراں سے باہر نکل آیا ۔", label=1),
    InputExample(guid=24, text_a="ہم رات کو رقص کر رہے ہیں ۔", label=1),
    InputExample(guid=25, text_a="ہرمن نے دھات کے فلیٹ پر ہتھوڑا مارا ۔", label=1),
    InputExample(guid=26, text_a="ناقدین نے اسٹیج سے باہر ڈرامے کو ہنس دیا ۔", label=1),
    InputExample(guid=27, text_a="تالاب ٹھوس طور پر جم گیا ۔", label=1),
    InputExample(guid=28, text_a="بل کمرے سے باہر نکل آیا ۔", label=1),
    InputExample(guid=29, text_a="باغبان نے پھولوں کو چپٹا پانی دیا ۔", label=1),
    InputExample(guid=30, text_a="باغبان نے پھولوں کو پانی دیا ۔", label=1),
    InputExample(guid=31, text_a="بل نے باتھ ٹب کو ٹکڑوں میں توڑ دیا ۔", label=1),    
]


In [6]:
# ==============================
# Define Classes
# ==============================
classes = ["unacc", "acc"]
label_map = {"unacc": 0, "acc": 1}

# # Step 1: Use load_plm with 'roberta' to get the correct WrapperClass
_, _, _, WrapperClass = load_plm("roberta", "roberta-base")  # Just to get the wrapper

# # Step 2: Manually load XLM-RoBERTa model/tokenizer
model_name = "xlm-roberta-base"
tokenizer = XLMRobertaTokenizer.from_pretrained(model_name)
plm = XLMRobertaForMaskedLM.from_pretrained(model_name)

# ==============================
# Load Pretrained Language Model (mBERT)
# ==============================
# plm, tokenizer, model_config, WrapperClass = load_plm("bert", "bert-base-multilingual-cased")


C:\Users\stdFurqan\anaconda3\envs\py310\lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [7]:

# ==============================
# Define Prompt Template (Manual)
# ==============================
template = ManualTemplate(
    # text = '{"placeholder":"text_a"} هذه الجملة {"mask"}.',
    text='{"placeholder":"text_a"} یہ جملہ {"mask"} ہے۔',
    tokenizer=tokenizer,
)




templates = [
    ("P1", ManualTemplate(
        text='{"placeholder":"text_a"} یہ جملہ {"mask"} ہے۔',  # Keep as is
        tokenizer=tokenizer,
    )),
    ("P2", ManualTemplate(
        text='دی گئی عبارت: {"placeholder":"text_a"} کا مواد اور پیغام {"mask"} ہے۔',  # generalized for neg/pos
        tokenizer=tokenizer,
    )),
    ("P3", ManualTemplate(
        text='اس عبارت کا تجزیہ کریں: {"placeholder":"text_a"} اس کا احساس یا موڈ {"mask"} ہے۔',  # focuses on sentiment/intent
        tokenizer=tokenizer,
    )),
    ("P4", ManualTemplate(
        text='جملہ: {"placeholder":"text_a"} ہمیں بتاتا ہے کہ یہ مواد {"mask"} ہے۔',  # clear statement aligned with label
        tokenizer=tokenizer,
    )),
    ("P5", ManualTemplate(
        text='{"placeholder":"text_a"} اس عبارت کا مطلب {"mask"} ہے۔',
        tokenizer=tokenizer,
    )),
    ("P6", ManualTemplate(
        text='{"placeholder":"text_a"} اس معاملے میں حتمی رائے {"mask"}',
        tokenizer=tokenizer,
    )),
    ("P7", ManualTemplate(
        text='{"placeholder":"text_a"} اس مواد کی تشریح {"mask"}',
        tokenizer=tokenizer,
    )),
    ("P8", ManualTemplate(
        text='{"placeholder":"text_a"} اس حوالے سے فیصلہ {"mask"}',
        tokenizer=tokenizer,
    )),
    ("P9", ManualTemplate(
        text='{"placeholder":"text_a"} اس متن کی درجہ بندی {"mask"}',
        tokenizer=tokenizer,
    )),
    ("P10", ManualTemplate(
        text='{"placeholder":"text_a"} اس اظہار کا نتیجہ {"mask"}',
        tokenizer=tokenizer,
    )), 
]



# ==============================
# Define Verbalizer (Manual)
# ==============================
verbalizer = ManualVerbalizer(
    classes=classes,
    label_words = {
        "acc": ["صحیح","درست"], 
       "unacc": ["نا درست","غلط"]
    },
    tokenizer=tokenizer,
)

In [8]:
# ==============================
# Create Prompt Model
# ==============================
prompt_model = PromptForClassification(
    template=template,
    plm=plm,
    verbalizer=verbalizer
)

# ==============================
# Training loop with BalancedBatchSampler + random template switching
# ==============================
T = 20   # epochs
K = 1    # steps per prompt
batch_size = 8





prompt_model.train()
optimizer = AdamW(prompt_model.parameters(), lr=1e-5)
all_epoch_patterns = {}

for epoch in range(T):
    print(f"\n🟦 Epoch {epoch+1}/{T}")

    # Random initial template
    prompt_name, current_template = random.choice(templates)
    epoch_pattern = []

    # Create PromptDataLoader with BalancedBatchSampler
    sampler = BalancedBatchSampler(train_dataset, batch_size=batch_size)
    train_loader = PromptDataLoader(
        dataset=train_dataset,
        tokenizer=tokenizer,
        template=current_template,
        tokenizer_wrapper_class=WrapperClass,
        max_seq_length=128,
        batch_size=batch_size,
        batch_sampler=sampler,
        shuffle=False  # shuffle is ignored when batch_sampler is used
    )

    step_counter = 0
    epoch_loss = 0.0

    for batch in train_loader:
        # Move batch to device
        # batch = {k: v.to(device) for k, v in batch.items()}
    
        optimizer.zero_grad()
        logits = prompt_model(batch)
        loss = torch.nn.CrossEntropyLoss()(logits, batch['label'])
        # loss = criterion(logits, batch["label"])
        
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_pattern.append(prompt_name)

        step_counter += 1

        # Switch template every K steps
        if step_counter % K == 0:
            prompt_name, current_template = random.choice(templates)

            # Rebuild PromptDataLoader with new template but same sampler
            train_loader = PromptDataLoader(
                dataset=train_dataset,
                tokenizer=tokenizer,
                template=current_template,
                tokenizer_wrapper_class=WrapperClass,
                max_seq_length=128,
                batch_size=batch_size,
                batch_sampler=sampler,
                shuffle=False
            )

    all_epoch_patterns[f"epoch_{epoch+1}"] = epoch_pattern
    print(f"Epoch {epoch+1} Loss: {epoch_loss:.4f}")
    print(f"Prompt pattern: {epoch_pattern}")



🟦 Epoch 1/20


tokenizing: 32it [00:00, 2582.35it/s]
tokenizing: 32it [00:00, 2779.93it/s]
tokenizing: 32it [00:00, 2368.45it/s]
tokenizing: 32it [00:00, 2042.76it/s]
tokenizing: 32it [00:00, 2031.42it/s]


Epoch 1 Loss: 12.0558
Prompt pattern: ['P2', 'P1', 'P5', 'P4']

🟦 Epoch 2/20


tokenizing: 32it [00:00, 2278.70it/s]
tokenizing: 32it [00:00, 2097.94it/s]
tokenizing: 32it [00:00, 2593.13it/s]
tokenizing: 32it [00:00, 1937.55it/s]
tokenizing: 32it [00:00, 2557.70it/s]


Epoch 2 Loss: 2.8763
Prompt pattern: ['P3', 'P2', 'P9', 'P2']

🟦 Epoch 3/20


...

Epoch 18 Loss: 1.0855
Prompt pattern: ['P1', 'P6', 'P7', 'P5']

🟦 Epoch 19/20


tokenizing: 32it [00:00, 2786.33it/s]
tokenizing: 32it [00:00, 4000.77it/s]
tokenizing: 32it [00:00, 5333.51it/s]
tokenizing: 32it [00:00, 2908.48it/s]
tokenizing: 32it [00:00, 4569.89it/s]


Epoch 19 Loss: 0.9758
Prompt pattern: ['P4', 'P10', 'P6', 'P4']

🟦 Epoch 20/20


tokenizing: 32it [00:00, 4129.65it/s]
tokenizing: 32it [00:00, 3504.38it/s]
tokenizing: 32it [00:00, 2916.95it/s]
tokenizing: 32it [00:00, 3323.21it/s]
tokenizing: 32it [00:00, 2850.06it/s]

Epoch 20 Loss: 0.8804
Prompt pattern: ['P7', 'P8', 'P3', 'P5']


In [9]:
# ==============================
# Load Evaluation Dataset
# ==============================
df = pd.read_csv(r"C:\Users\stdFurqan\Desktop\paft\cola_dataset\final_ColA_Dev_Urdu_labeled - Sheet1.csv")
eval_dataset = [
    InputExample(guid=i, text_a=row['Urdu Sentence'], label=label_map[row['label']])
    for i, row in df.iterrows()
]


eval_loader = PromptDataLoader(
    dataset=eval_dataset,
    tokenizer=tokenizer,
    template=template,
    tokenizer_wrapper_class=WrapperClass,
    max_seq_length=128,
    batch_size=8,
    shuffle=False
)

# ==============================
# Evaluate Model
# ==============================
prompt_model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in eval_loader:
        logits = prompt_model(batch)
        preds = torch.argmax(logits, dim=-1)
        all_preds.extend(preds.cpu().tolist())
        all_labels.extend(batch['label'].cpu().tolist())

# ==============================
# Print Classification Report
# ==============================
print("\n📊 Fine-tuned Classification Report:")
print(classification_report(all_labels, all_preds, target_names=classes, digits=4))


tokenizing: 1043it [00:00, 4192.74it/s]



📊 Fine-tuned Classification Report:
              precision    recall  f1-score   support

       unacc     0.4502    0.3509    0.3944       322
         acc     0.7361    0.8086    0.7707       721

    accuracy                         0.6673      1043
   macro avg     0.5932    0.5798    0.5825      1043
weighted avg     0.6478    0.6673    0.6545      1043



In [ ]:
roberta
📊 Fine-tuned Classification Report:
              precision    recall  f1-score   support

       unacc     0.4502    0.3509    0.3944       322
         acc     0.7361    0.8086    0.7707       721

    accuracy                         0.6673      1043
   macro avg     0.5932    0.5798    0.5825      1043
weighted avg     0.6478    0.6673    0.6545      1043

In [ ]:
mbert
📊 Fine-tuned Classification Report:
              precision    recall  f1-score   support

       unacc     0.4346    0.2888    0.3470       322
         acc     0.7238    0.8322    0.7742       721

    accuracy                         0.6644      1043
   macro avg     0.5792    0.5605    0.5606      1043
weighted avg     0.6345    0.6644    0.6423      1043